Report for Task A based on Deep Neural Network using H2O and python.


This imports the h2o library and connects with the h2o server. The server has to be running.

In [63]:
import h2o

# switch to java 8 "sdk use java 8.0.452-tem"
# java -jar h2o.jar
h2o.init(ip="localhost", port="54321")
print(h2o.__version__)

Checking whether there is an H2O instance running at http://localhost:54321. connected.
Attempting to start a local H2O server...
  Java Version: openjdk version "21.0.6" 2025-01-21 LTS; OpenJDK Runtime Environment Temurin-21.0.6+7 (build 21.0.6+7-LTS); OpenJDK 64-Bit Server VM Temurin-21.0.6+7 (build 21.0.6+7-LTS, mixed mode, sharing)
  Starting server from /home/adam/GithubProjects/pndc-lab/.venv/lib/python3.12/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpfebgp9_c
  JVM stdout: /tmp/tmpfebgp9_c/h2o_adam_started_from_python.out
  JVM stderr: /tmp/tmpfebgp9_c/h2o_adam_started_from_python.err
  Server is running at http://127.0.0.1:54333
Connecting to H2O server at http://127.0.0.1:54333 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Europe/Warsaw
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.7
H2O_cluster_version_age:,1 month and 26 days
H2O_cluster_name:,H2O_from_python_adam_e9snn7
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,4.820 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


3.46.0.7


Helper function to build and test deep neural network model

In [75]:
from h2o.estimators.deeplearning import H2ODeepLearningEstimator

metrics = []

def run_experiment(dataset, features, target, hidden, nfolds, activation):
    train, test = dataset.split_frame(ratios=[0.8], seed=42)
    dnn = H2ODeepLearningEstimator(activation=activation, hidden=hidden, nfolds=nfolds, seed=42, reproducible=True)
    dnn.train(x=features, y=target, training_frame=train, validation_frame=test)
    #print(dnn.model_performance().mean_per_class_error(train=True))
    #print(dnn.run_time / 1000)
    print(dnn.show())
    metrics.append([activation, "None" if nfolds == 0 else f"{nfolds}-folds", hidden, 
                    f"{dnn.mean_per_class_error(train=True)[0][1] * 100}%", 
                    f"{dnn.mean_per_class_error(valid=True)[0][1] * 100}%", 
                    f"{dnn.mean_per_class_error(xval=True)[0][1] * 100}%" if nfolds != 0 else "-", 
                    f"{dnn.run_time / 1000} s"])

Sonar dataset

In [65]:
col_names = [f"x_{i}" for i in range(60)]
col_names.append("class")
sonar = h2o.import_file(path="datasets/sonar/sonar.all-data", col_names=col_names)

features = col_names[:-1]
target = "class"
print(sonar)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
   x_0     x_1     x_2     x_3     x_4     x_5     x_6     x_7     x_8     x_9    x_10    x_11    x_12    x_13    x_14    x_15    x_16    x_17    x_18    x_19    x_20    x_21    x_22    x_23    x_24    x_25    x_26    x_27    x_28    x_29    x_30    x_31    x_32    x_33    x_34    x_35    x_36    x_37    x_38    x_39    x_40    x_41    x_42    x_43    x_44    x_45    x_46    x_47    x_48    x_49    x_50    x_51    x_52    x_53    x_54    x_55    x_56    x_57    x_58    x_59  class
0.02    0.0371  0.0428  0.0207  0.0954  0.0986  0.1539  0.1601  0.3109  0.2111  0.1609  0.1582  0.2238  0.0645  0.066   0.2273  0.31    0.2999  0.5078  0.4797  0.5783  0.5071  0.4328  0.555   0.6711  0.6415  0.7104  0.808   0.6791  0.3857  0.1307  0.2604  0.5121  0.7547  0.8537  0.8507  0.6692  0.6097  0.4943  0.2744  0.051   0.2834  0.2825  0.4256  0.2641  0.1386  0.1051  0.1343  0.0383  0.0324  0.0232  0.0027  0.0

Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,], cross folds = 0, activation function = "rectifier"

In [60]:
print ((0.43756707326796185 + 0.27471198975963596)/2)

0.3561395315137989


In [76]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_106


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,680 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ----------------------  -------------------
    1        60       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.001559454756982935   0.0009270920418202877  0.0         -0.01678416559152538  0.17648959159851074  0.492606163333497       0.12412995100021362
    3        5        Rectifier  0.0        0.0   0.0   0.0009398416453041136  0.0004263881128281355  0.0         -0.11639514479786157  0.46815311908721924  0.8774447056570367      0.02689208835363388
    4        5        Rectifier  0.0        0.0   0.0   0.0030883694236399604  0.003574732691049576   0.0         0.15138846710324289   0.3587327003479004   1.1962029792582993      0.4891887903213501
    5        2        Softmax               0.0   0.0   0.0015863887732848526  0.0014392049051821232  0.0         1.6811171561479568    0.914656400680542    -5.551115123125783e-17  0.1872606873512268

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.19188259874750094
RMSE: 0.43804406028104176
LogLoss: 0.5652672288689073
Mean Per-Class Error: 0.2851656947802588
AUC: 0.7835300810695491
AUCPR: 0.7293018687300358
Gini: 0.5670601621390983

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.30830047376674424
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      45   44   0.4944   (44.0/89.0)
R      6    73   0.0759   (6.0/79.0)
Total  51   117  0.2976   (50.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.3083       0.744898  113
max f2                       0.247859     0.848214  125
max f0point5                 0.634905     0.721649  49
max accuracy                 0.553468     0.720238  74
max precision                0.715163     1         0
max recall                   0.0875618    1         151
max specificity              0.715163     1         0
max absolute_mcc             0.3083       0.466406  113
max min_per_class_accuracy   0.520033     0.708861  77
max mean_per_class_accuracy  0.437567     0.725288  92
max tns                      0.715163     89        0
max fns                      0.715163     78        0
max fps                      0.0162763    89        161
max tps                      0.0875618    79        151
max tnr                      0.715163     1         0
max fnr                      0.715163     0.987342  0
max fpr                      0.0162763    1         161
max tpr                      0.0875618    1         151

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 44.91 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0119048                   0.711176           2.12658   2.12658            1                0.714045   1                           0.714045            0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 0, activation function = "rectifier"


In [78]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_127


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,102 weights/biases, 41.2 KB, 1,680 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ----------------------  -------------------  ----------------------  --------------------
    1        60       Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.0014184095170639921  0.0006653934251517057  0.0         -0.0016727960794620837  0.16495448350906372  0.50064894060232        0.02519838511943817
    3        20       Rectifier  0.0        0.0   0.0   0.0011516788242806797  0.0007868160028010607  0.0         0.009429035882349127    0.22134089469909668  1.010986751889036       0.027634017169475555
    4        20       Rectifier  0.0        0.0   0.0   0.0024827512600859337  0.02082657814025879    0.0         0.006575527810528001    0.21393156051635742  0.9949089547141211      0.019399069249629974
    5        2        Softmax               0.0   0.0   0.001929180858132895   0.003024900332093239   0.0         0.03699483573436737     1.1441965103149414   5.0306980803327406e-17  0.014902666211128235

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.027584886969943018
RMSE: 0.16608698615467443
LogLoss: 0.11463414319521362
Mean Per-Class Error: 0.02247191011235955
AUC: 0.996159863461812
AUCPR: 0.9953758812491246
Gini: 0.9923197269236239

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5484170215316417
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      85   4    0.0449   (4.0/89.0)
R      0    79   0        (0.0/79.0)
Total  85   83   0.0238   (4.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.548417     0.975309  82
max f2                       0.548417     0.989975  82
max f0point5                 0.698978     0.979112  75
max accuracy                 0.604744     0.97619   80
max precision                0.999823     1         0
max recall                   0.548417     1         82
max specificity              0.999823     1         0
max absolute_mcc             0.548417     0.95343   82
max min_per_class_accuracy   0.606711     0.966292  79
max mean_per_class_accuracy  0.548417     0.977528  82
max tns                      0.999823     89        0
max fns                      0.999823     78        0
max fps                      1.32138e-06  89        167
max tps                      0.548417     79        82
max tnr                      0.999823     1         0
max fnr                      0.999823     0.987342  0
max fpr                      1.32138e-06  1         167
max tpr                      0.548417     1         82

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 48.17 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999576           2.12658  2.12658            1                0.999706     1                           0.999706            0.0253165       0.0253165 

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 0, activation function = "rectifier"

In [79]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_148


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,502 weights/biases, 329.9 KB, 840 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight             weight_rms           mean_bias                bias_rms
--  -------  -------  ---------  ---------  ----  ----  --------------------  ---------------------  ----------  ----------------------  -------------------  -----------------------  ----------------------
    1        60       Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.002448070572616416  0.0011453917250037193  0.0         0.003252465802664195    0.11115986108779907  0.49674366445166335      0.010552480816841125
    3        100      Rectifier  0.0        0.0   0.0   0.002707152927032439  0.0018438687548041344  0.0         -0.0020940863255340444  0.1012243926525116   0.9951804930319187       0.006924398243427277
    4        100      Rectifier  0.0        0.0   0.0   0.028738380738571868  0.1379932165145874     0.0         -0.0005828282664311701  0.10069042444229126  0.9983629786125476       0.002809491939842701
    5        2        Softmax               0.0   0.0   0.021956822120118888  0.13766086101531982    0.0         -0.007474556691595353   0.5508320331573486   -1.2197274440461925e-18  0.00024074350949376822

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.01969414696112127
RMSE: 0.1403358363395511
LogLoss: 0.07420767880557251
Mean Per-Class Error: 0.017565068980230407
AUC: 0.9987199544872706
AUCPR: 0.998571345320299
Gini: 0.9974399089745412

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6780486350935179
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      87   2    0.0225   (2.0/89.0)
R      1    78   0.0127   (1.0/79.0)
Total  88   80   0.0179   (3.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.678049     0.981132  79
max f2                       0.447293     0.989975  82
max f0point5                 0.945225     0.986667  73
max accuracy                 0.891779     0.982143  77
max precision                1            1         0
max recall                   0.447293     1         82
max specificity              1            1         0
max absolute_mcc             0.678049     0.964253  79
max min_per_class_accuracy   0.678049     0.977528  79
max mean_per_class_accuracy  0.678049     0.982435  79
max tns                      1            89        0
max fns                      1            78        0
max fps                      3.32032e-09  89        167
max tps                      0.447293     79        82
max tnr                      1            1         0
max fnr                      1            0.987342  0
max fpr                      3.32032e-09  1         167
max tpr                      0.447293     1         82

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 50.56 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   1                  2.12658  2.12658            1                1            1                           1                   0.0253165       0.0253

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 5, activation function = "rectifier"


In [80]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_161


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,848 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ----------------------  -------------------
    1        60       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.001713738647425392   0.0010229605250060558  0.0         -0.01696698335465044  0.17652469873428345  0.4897004436024798      0.12273091077804565
    3        5        Rectifier  0.0        0.0   0.0   0.0011018532770685852  0.0005585390608757734  0.0         -0.11575018931180239  0.467958927154541    0.8773680550488978      0.02741599828004837
    4        5        Rectifier  0.0        0.0   0.0   0.0030854031641501935  0.003537634387612343   0.0         0.1496608740091324    0.3608980178833008   1.202863601443489       0.4960860013961792
    5        2        Softmax               0.0   0.0   0.0015969251864589751  0.001426855567842722   0.0         1.6811175704002381    0.9172227382659912   -5.551115123125783e-17  0.18831920623779297

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.18223224074112446
RMSE: 0.4268866837242929
LogLoss: 0.5419850649405522
Mean Per-Class Error: 0.2683117621959892
AUC: 0.8077087185322144
AUCPR: 0.7659378754585661
Gini: 0.6154174370644288

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33737275990412413
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      48   41   0.4607   (41.0/89.0)
R      6    73   0.0759   (6.0/79.0)
Total  54   114  0.2798   (47.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.337373     0.756477  111
max f2                       0.282585     0.857788  120
max f0point5                 0.652808     0.736677  57
max accuracy                 0.5144       0.738095  82
max precision                0.757113     1         0
max recall                   0.103388     1         149
max specificity              0.757113     1         0
max absolute_mcc             0.337373     0.495214  111
max min_per_class_accuracy   0.571713     0.721519  78
max mean_per_class_accuracy  0.452777     0.742853  92
max tns                      0.757113     89        0
max fns                      0.757113     78        0
max fps                      0.0149589    89        161
max tps                      0.103388     79        149
max tnr                      0.757113     1         0
max fnr                      0.757113     0.987342  0
max fpr                      0.0149589    1         161
max tpr                      0.103388     1         149

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 46.98 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0119048                   0.751181           2.12658   2.12658            1                0.756802   1                           0.756802            0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 5, activation function = "rectifier"


In [81]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_320


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,102 weights/biases, 41.2 KB, 1,848 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  --------------------
    1        60       Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.0015385607572534355  0.000720346812158823   0.0         -0.001756984709839647  0.16516298055648804  0.5007774606739016     0.025136515498161316
    3        20       Rectifier  0.0        0.0   0.0   0.0011768024454431724  0.0007805891800671816  0.0         0.00955651056021452    0.2214859127998352   1.011213558004725      0.027949146926403046
    4        20       Rectifier  0.0        0.0   0.0   0.002495302957140666   0.020933307707309723   0.0         0.006532524077338167   0.2140679955482483   0.994859388861207      0.019818998873233795
    5        2        Softmax               0.0   0.0   0.00200612103799358    0.0031102001667022705  0.0         0.03699476746842265    1.144667148590088    5.377642775528102e-17  0.01524781808257103

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02892864849769829
RMSE: 0.1700842394159385
LogLoss: 0.11397104522371825
Mean Per-Class Error: 0.01194709145214052
AUC: 0.9982932726496943
AUCPR: 0.9980347668663914
Gini: 0.9965865452993885

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7890691171117747
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      88   1    0.0112   (1.0/89.0)
R      1    78   0.0127   (1.0/79.0)
Total  89   79   0.0119   (2.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.789069     0.987342  78
max f2                       0.716393     0.989975  82
max f0point5                 0.789069     0.987342  78
max accuracy                 0.789069     0.988095  78
max precision                0.99986      1         0
max recall                   0.716393     1         82
max specificity              0.99986      1         0
max absolute_mcc             0.789069     0.976106  78
max min_per_class_accuracy   0.789069     0.987342  78
max mean_per_class_accuracy  0.789069     0.988053  78
max tns                      0.99986      89        0
max fns                      0.99986      78        0
max fps                      8.34374e-07  89        167
max tps                      0.716393     79        82
max tnr                      0.99986      1         0
max fnr                      0.99986      0.987342  0
max fpr                      8.34374e-07  1         167
max tpr                      0.716393     1         82

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 51.27 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999722           2.12658  2.12658            1                0.99984      1                           0.99984             0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 5, activation function = "rectifier"

In [82]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_483


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,502 weights/biases, 329.9 KB, 840 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        60       Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.00259032053941822    0.001152674201875925  0.0         0.0032336403432536827   0.11120826005935669  0.4965094780712604      0.011110015213489532
    3        100      Rectifier  0.0        0.0   0.0   0.0028366732829774263  0.001843986101448536  0.0         -0.002089339655171898   0.10125362873077393  0.9952352928945891      0.006813514977693558
    4        100      Rectifier  0.0        0.0   0.0   0.024492461352405372   0.11916416883468628   0.0         -0.0006871483343387809  0.10072726011276245  0.9982590791391017      0.0030125873163342476
    5        2        Softmax               0.0   0.0   0.021967239729710854   0.13758432865142822   0.0         -0.00747455102740787    0.5508298873901367   -7.589415207398531e-19  0.0005381270311772823

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.005908388166383023
RMSE: 0.07686604039745396
LogLoss: 0.029653551075837833
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5297428236582055
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      89   0    0        (0.0/89.0)
R      0    79   0        (0.0/79.0)
Total  89   79   0        (0.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.529743     1         78
max f2                       0.529743     1         78
max f0point5                 0.529743     1         78
max accuracy                 0.529743     1         78
max precision                1            1         0
max recall                   0.529743     1         78
max specificity              1            1         0
max absolute_mcc             0.529743     1         78
max min_per_class_accuracy   0.529743     1         78
max mean_per_class_accuracy  0.529743     1         78
max tns                      1            89        0
max fns                      1            78        0
max fps                      4.71799e-11  89        167
max tps                      0.529743     79        78
max tnr                      1            1         0
max fnr                      1            0.987342  0
max fpr                      4.71799e-11  1         167
max tpr                      0.529743     1         78

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 45.17 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   1                  2.12658  2.12658            1                1            1                           1                   0.0253165       0.0253165                  112.658  112.658            0.0253165
2     

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 10, activation function = "rectifier"


In [83]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="rectifier")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_566


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,848 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight           weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  --------------------  -------------------  ----------------------  -------------------
    1        60       Input      0.0
    2        5        Rectifier  0.0        0.0   0.0   0.001713738647425392   0.0010229605250060558  0.0         -0.01696698335465044  0.17652469873428345  0.4897004436024798      0.12273091077804565
    3        5        Rectifier  0.0        0.0   0.0   0.0011018532770685852  0.0005585390608757734  0.0         -0.11575018931180239  0.467958927154541    0.8773680550488978      0.02741599828004837
    4        5        Rectifier  0.0        0.0   0.0   0.0030854031641501935  0.003537634387612343   0.0         0.1496608740091324    0.3608980178833008   1.202863601443489       0.4960860013961792
    5        2        Softmax               0.0   0.0   0.0015969251864589751  0.001426855567842722   0.0         1.6811175704002381    0.9172227382659912   -5.551115123125783e-17  0.18831920623779297

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.18223224074112446
RMSE: 0.4268866837242929
LogLoss: 0.5419850649405522
Mean Per-Class Error: 0.2683117621959892
AUC: 0.8077087185322144
AUCPR: 0.7659378754585661
Gini: 0.6154174370644288

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.33737275990412413
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      48   41   0.4607   (41.0/89.0)
R      6    73   0.0759   (6.0/79.0)
Total  54   114  0.2798   (47.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.337373     0.756477  111
max f2                       0.282585     0.857788  120
max f0point5                 0.652808     0.736677  57
max accuracy                 0.5144       0.738095  82
max precision                0.757113     1         0
max recall                   0.103388     1         149
max specificity              0.757113     1         0
max absolute_mcc             0.337373     0.495214  111
max min_per_class_accuracy   0.571713     0.721519  78
max mean_per_class_accuracy  0.452777     0.742853  92
max tns                      0.757113     89        0
max fns                      0.757113     78        0
max fps                      0.0149589    89        161
max tps                      0.103388     79        149
max tnr                      0.757113     1         0
max fnr                      0.757113     0.987342  0
max fpr                      0.0149589    1         161
max tpr                      0.103388     1         149

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 46.98 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0119048                   0.751181           2.12658   2.12658            1                0.756802   1                           0.756802            0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "rectifier"

In [84]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_838


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,102 weights/biases, 41.2 KB, 1,848 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias              bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  ---------------------  --------------------
    1        60       Input      0.0
    2        20       Rectifier  0.0        0.0   0.0   0.0015385607572534355  0.000720346812158823   0.0         -0.001756984709839647  0.16516298055648804  0.5007774606739016     0.025136515498161316
    3        20       Rectifier  0.0        0.0   0.0   0.0011768024454431724  0.0007805891800671816  0.0         0.00955651056021452    0.2214859127998352   1.011213558004725      0.027949146926403046
    4        20       Rectifier  0.0        0.0   0.0   0.002495302957140666   0.020933307707309723   0.0         0.006532524077338167   0.2140679955482483   0.994859388861207      0.019818998873233795
    5        2        Softmax               0.0   0.0   0.00200612103799358    0.0031102001667022705  0.0         0.03699476746842265    1.144667148590088    5.377642775528102e-17  0.01524781808257103

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02892864849769829
RMSE: 0.1700842394159385
LogLoss: 0.11397104522371825
Mean Per-Class Error: 0.01194709145214052
AUC: 0.9982932726496943
AUCPR: 0.9980347668663914
Gini: 0.9965865452993885

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.7890691171117747
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      88   1    0.0112   (1.0/89.0)
R      1    78   0.0127   (1.0/79.0)
Total  89   79   0.0119   (2.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.789069     0.987342  78
max f2                       0.716393     0.989975  82
max f0point5                 0.789069     0.987342  78
max accuracy                 0.789069     0.988095  78
max precision                0.99986      1         0
max recall                   0.716393     1         82
max specificity              0.99986      1         0
max absolute_mcc             0.789069     0.976106  78
max min_per_class_accuracy   0.789069     0.987342  78
max mean_per_class_accuracy  0.789069     0.988053  78
max tns                      0.99986      89        0
max fns                      0.99986      78        0
max fps                      8.34374e-07  89        167
max tps                      0.716393     79        82
max tnr                      0.99986      1         0
max fnr                      0.99986      0.987342  0
max fpr                      8.34374e-07  1         167
max tpr                      0.716393     1         82

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 51.27 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999722           2.12658  2.12658            1                0.99984      1                           0.99984             0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 10, activation function = "rectifier"


In [85]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="rectifier")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748030642422_1112


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,502 weights/biases, 329.9 KB, 840 training samples, mini-batch size 1
    layer    units    type       dropout    l1    l2    mean_rate              rate_rms              momentum    mean_weight             weight_rms           mean_bias               bias_rms
--  -------  -------  ---------  ---------  ----  ----  ---------------------  --------------------  ----------  ----------------------  -------------------  ----------------------  ---------------------
    1        60       Input      0.0
    2        100      Rectifier  0.0        0.0   0.0   0.00259032053941822    0.001152674201875925  0.0         0.0032336403432536827   0.11120826005935669  0.4965094780712604      0.011110015213489532
    3        100      Rectifier  0.0        0.0   0.0   0.0028366732829774263  0.001843986101448536  0.0         -0.002089339655171898   0.10125362873077393  0.9952352928945891      0.006813514977693558
    4        100      Rectifier  0.0        0.0   0.0   0.024492461352405372   0.11916416883468628   0.0         -0.0006871483343387809  0.10072726011276245  0.9982590791391017      0.0030125873163342476
    5        2        Softmax               0.0   0.0   0.021967239729710854   0.13758432865142822   0.0         -0.00747455102740787    0.5508298873901367   -7.589415207398531e-19  0.0005381270311772823

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.005908388166383023
RMSE: 0.07686604039745396
LogLoss: 0.029653551075837833
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5297428236582055
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      89   0    0        (0.0/89.0)
R      0    79   0        (0.0/79.0)
Total  89   79   0        (0.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.529743     1         78
max f2                       0.529743     1         78
max f0point5                 0.529743     1         78
max accuracy                 0.529743     1         78
max precision                1            1         0
max recall                   0.529743     1         78
max specificity              1            1         0
max absolute_mcc             0.529743     1         78
max min_per_class_accuracy   0.529743     1         78
max mean_per_class_accuracy  0.529743     1         78
max tns                      1            89        0
max fns                      1            78        0
max fps                      4.71799e-11  89        167
max tps                      0.529743     79        78
max tnr                      1            1         0
max fnr                      1            0.987342  0
max fpr                      4.71799e-11  1         167
max tpr                      0.529743     1         78

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 45.17 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   1                  2.12658  2.12658            1                1            1                           1                   0.0253165       0.0253165                  112.658  112.658            0.0253165
2    

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 0, activation function = "tanh"


In [13]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1153


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,680 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  --------------------
    1        60       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.002712150146641458  0.00150197371840477    0.0         -0.02362282778087926  0.18349027633666992  0.015479677376569755   0.05787169933319092
    3        5        Tanh     0.0        0.0   0.0   0.004626283925026655  0.0037664100527763367  0.0         -0.03942650854587555  0.4685739278793335   0.03334589617003594    0.046395450830459595
    4        5        Tanh     0.0        0.0   0.0   0.026629312126897276  0.038649797439575195   0.0         0.03938303232192993   0.3998473882675171   0.05544716087343508    0.1040254533290863
    5        2        Softmax             0.0   0.0   0.003579130582511425  0.0011311699636280537  0.0         1.6811172902584075    0.963719367980957    1.734723475976807e-17  0.033019766211509705

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.14945424273758487
RMSE: 0.38659312298278775
LogLoss: 0.46613621349921047
Mean Per-Class Error: 0.18553548570615844
AUC: 0.8613284027876547
AUCPR: 0.8693778592782194
Gini: 0.7226568055753093

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.45809299866262665
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      74   15   0.1685   (15.0/89.0)
R      16   63   0.2025   (16.0/79.0)
Total  90   78   0.1845   (31.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.458093     0.802548  77
max f2                       0.222795     0.844749  121
max f0point5                 0.6637       0.813559  53
max accuracy                 0.458093     0.815476  77
max precision                0.916537     1         0
max recall                   0.107585     1         161
max specificity              0.916537     1         0
max absolute_mcc             0.458093     0.629422  77
max min_per_class_accuracy   0.442399     0.797753  81
max mean_per_class_accuracy  0.458093     0.814465  77
max tns                      0.916537     89        0
max fns                      0.916537     78        0
max fps                      0.0965751    89        167
max tps                      0.107585     79        161
max tnr                      0.916537     1         0
max fnr                      0.916537     0.987342  0
max fpr                      0.0965751    1         167
max tpr                      0.107585     1         161

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 46.53 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0119048                   0.908906           2.12658   2.12658            1                0.914782  1                           0.914782            0.0253165       0.0253165                  112.658   112.658  

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 0, activation function = "tanh"


In [14]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1174


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,680 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight           weight_rms           mean_bias              bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  --------------------  -------------------  ---------------------  --------------------
    1        60       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.002712150146641458  0.00150197371840477    0.0         -0.02362282778087926  0.18349027633666992  0.015479677376569755   0.05787169933319092
    3        5        Tanh     0.0        0.0   0.0   0.004626283925026655  0.0037664100527763367  0.0         -0.03942650854587555  0.4685739278793335   0.03334589617003594    0.046395450830459595
    4        5        Tanh     0.0        0.0   0.0   0.026629312126897276  0.038649797439575195   0.0         0.03938303232192993   0.3998473882675171   0.05544716087343508    0.1040254533290863
    5        2        Softmax             0.0   0.0   0.003579130582511425  0.0011311699636280537  0.0         1.6811172902584075    0.963719367980957    1.734723475976807e-17  0.033019766211509705

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.14945424273758487
RMSE: 0.38659312298278775
LogLoss: 0.46613621349921047
Mean Per-Class Error: 0.18553548570615844
AUC: 0.8613284027876547
AUCPR: 0.8693778592782194
Gini: 0.7226568055753093

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.45809299866262665
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      74   15   0.1685   (15.0/89.0)
R      16   63   0.2025   (16.0/79.0)
Total  90   78   0.1845   (31.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.458093     0.802548  77
max f2                       0.222795     0.844749  121
max f0point5                 0.6637       0.813559  53
max accuracy                 0.458093     0.815476  77
max precision                0.916537     1         0
max recall                   0.107585     1         161
max specificity              0.916537     1         0
max absolute_mcc             0.458093     0.629422  77
max min_per_class_accuracy   0.442399     0.797753  81
max mean_per_class_accuracy  0.458093     0.814465  77
max tns                      0.916537     89        0
max fns                      0.916537     78        0
max fps                      0.0965751    89        167
max tps                      0.107585     79        161
max tnr                      0.916537     1         0
max fnr                      0.916537     0.987342  0
max fpr                      0.0965751    1         167
max tpr                      0.107585     1         161

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 46.53 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0119048                   0.908906           2.12658   2.12658            1                0.914782  1                           0.914782            0.0253165       0.0253165                  112.658   112.658  

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 0, activation function = "tanh"


In [15]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[20,20,20], nfolds=0, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1195


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,102 weights/biases, 41.1 KB, 1,680 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  -----------------------  ---------------------
    1        60       Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.0031009010916750413  0.0014736154116690159  0.0         0.0005154421842113758  0.16577595472335815  -0.0057282630751689595   0.03626415133476257
    3        20       Tanh     0.0        0.0   0.0   0.003888234206242487   0.0025008348748087883  0.0         0.002518447790753271   0.2241702675819397   0.0009604297782492881    0.04705968499183655
    4        20       Tanh     0.0        0.0   0.0   0.00307328522685566    0.0019278200343251228  0.0         0.010390526042028796   0.21478384733200073  0.00019744079485713024   0.02661243826150894
    5        2        Softmax             0.0   0.0   0.0036264889524318276  0.0010163276456296444  0.0         0.036994884721934795   1.1550850868225098   -1.6263032587282567e-18  0.0004949062131345272

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02334263029808408
RMSE: 0.15278295159501298
LogLoss: 0.10593629416207842
Mean Per-Class Error: 0.011235955056179775
AUC: 0.999146636324847
AUCPR: 0.9990445486586621
Gini: 0.998293272649694

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.49773515581748046
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      87   2    0.0225   (2.0/89.0)
R      0    79   0        (0.0/79.0)
Total  87   81   0.0119   (2.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.497735     0.9875    80
max f2                       0.497735     0.994962  80
max f0point5                 0.654627     0.992167  75
max accuracy                 0.497735     0.988095  80
max precision                0.999951     1         0
max recall                   0.497735     1         80
max specificity              0.999951     1         0
max absolute_mcc             0.497735     0.976418  80
max min_per_class_accuracy   0.56032      0.977528  79
max mean_per_class_accuracy  0.497735     0.988764  80
max tns                      0.999951     89        0
max fns                      0.999951     78        0
max fps                      5.94512e-06  89        167
max tps                      0.497735     79        80
max tnr                      0.999951     1         0
max fnr                      0.999951     0.987342  0
max fpr                      5.94512e-06  1         167
max tpr                      0.497735     1         80

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 47.40 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999833           2.12658  2.12658            1                0.999898     1                           0.999898            0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 0, activation function = "tanh"


In [16]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[100,100,100], nfolds=0, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1216


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,502 weights/biases, 329.9 KB, 840 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight            weight_rms           mean_bias               bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ---------------------  -------------------  ----------------------  --------------------
    1        60       Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.005083566853936645  0.0023829536512494087  0.0         0.0033072398633347197  0.11160707473754883  0.00344201451361394     0.017270125448703766
    3        100      Tanh     0.0        0.0   0.0   0.009262118021189236  0.005369473248720169   0.0         0.0007029317853648536  0.10173389315605164  7.925446437821119e-05   0.03069639950990677
    4        100      Tanh     0.0        0.0   0.0   0.027345032959256787  0.047752588987350464   0.0         0.001131764501503858   0.10110127925872803  -0.0017475837716631982  0.019746750593185425
    5        2        Softmax             0.0   0.0   0.005054590434301644  0.0018239389173686504  0.0         -0.007474570479243994  0.5434746742248535   -8.673617379884035e-18  0.018364854156970978

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.041086033124105074
RMSE: 0.20269689964107757
LogLoss: 0.13113635481541044
Mean Per-Class Error: 0.0407481154885507
AUC: 0.9928886360403926
AUCPR: 0.9918913015960971
Gini: 0.9857772720807851

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6752828150252148
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      84   5    0.0562   (5.0/89.0)
R      2    77   0.0253   (2.0/79.0)
Total  86   82   0.0417   (7.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.675283     0.956522  81
max f2                       0.436156     0.972906  89
max f0point5                 0.896232     0.973333  73
max accuracy                 0.896232     0.958333  73
max precision                0.999985     1         0
max recall                   0.436156     1         89
max specificity              0.999985     1         0
max absolute_mcc             0.896232     0.917722  73
max min_per_class_accuracy   0.693902     0.955056  79
max mean_per_class_accuracy  0.675283     0.959252  81
max tns                      0.999985     89        0
max fns                      0.999985     78        0
max fps                      3.32186e-08  89        167
max tps                      0.436156     79        89
max tnr                      0.999985     1         0
max fnr                      0.999985     0.987342  0
max fpr                      3.32186e-08  1         167
max tpr                      0.436156     1         89

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 50.95 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0119048                   0.999949           2.12658   2.12658            1                0.999975     1                           0.999975            0.0253165       0.0253165                  112

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 5, activation function = "tanh"


In [17]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=5, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1229


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,848 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  --------------------  --------------------
    1        60       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.0027169045708918325  0.0015208953991532326  0.0         -0.023698948222057273  0.18388843536376953  0.015306471072023042  0.06186409294605255
    3        5        Tanh     0.0        0.0   0.0   0.004841180955991149   0.004191089421510696   0.0         -0.03935050696134567   0.46970808506011963  0.0319510136990834    0.049038395285606384
    4        5        Tanh     0.0        0.0   0.0   0.02862795041175559    0.0436774343252182     0.0         0.04336357787251473    0.40053725242614746  0.05858446937420161   0.11086133122444153
    5        2        Softmax             0.0   0.0   0.003637024899944663   0.0012923544272780418  0.0         1.6811173975467681     0.9655880928039551   2.42861286636753e-17  0.03753940761089325

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.14209526860272684
RMSE: 0.3769552607441987
LogLoss: 0.4472297131145348
Mean Per-Class Error: 0.1792063717821078
AUC: 0.8718532214478737
AUCPR: 0.8779719303544561
Gini: 0.7437064428957474

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4606250882069269
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      74   15   0.1685   (15.0/89.0)
R      15   64   0.1899   (15.0/79.0)
Total  89   79   0.1786   (30.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.460625     0.810127  78
max f2                       0.223516     0.850575  118
max f0point5                 0.668388     0.825083  55
max accuracy                 0.460625     0.821429  78
max precision                0.926492     1         0
max recall                   0.100605     1         161
max specificity              0.926492     1         0
max absolute_mcc             0.460625     0.641587  78
max min_per_class_accuracy   0.460625     0.810127  78
max mean_per_class_accuracy  0.460625     0.820794  78
max tns                      0.926492     89        0
max fns                      0.926492     78        0
max fps                      0.0874641    89        167
max tps                      0.100605     79        161
max tnr                      0.926492     1         0
max fnr                      0.926492     0.987342  0
max fpr                      0.0874641    1         167
max tpr                      0.100605     1         161

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 46.78 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain       cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  ---------  -----------------  --------------------
1        0.0119048                   0.919765           2.12658   2.12658            1                0.924116  1                           0.924116            0.0253165       0.0253165                  112.658    112.

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 5, activation function = "tanh"


In [18]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[20,20,20], nfolds=5, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1390


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,102 weights/biases, 41.1 KB, 1,680 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  -----------------------  ---------------------
    1        60       Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.0031009010916750413  0.0014736154116690159  0.0         0.0005154421842113758  0.16577595472335815  -0.0057282630751689595   0.03626415133476257
    3        20       Tanh     0.0        0.0   0.0   0.003888234206242487   0.0025008348748087883  0.0         0.002518447790753271   0.2241702675819397   0.0009604297782492881    0.04705968499183655
    4        20       Tanh     0.0        0.0   0.0   0.00307328522685566    0.0019278200343251228  0.0         0.010390526042028796   0.21478384733200073  0.00019744079485713024   0.02661243826150894
    5        2        Softmax             0.0   0.0   0.0036264889524318276  0.0010163276456296444  0.0         0.036994884721934795   1.1550850868225098   -1.6263032587282567e-18  0.0004949062131345272

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.02334263029808408
RMSE: 0.15278295159501298
LogLoss: 0.10593629416207842
Mean Per-Class Error: 0.011235955056179775
AUC: 0.999146636324847
AUCPR: 0.9990445486586621
Gini: 0.998293272649694

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.49773515581748046
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      87   2    0.0225   (2.0/89.0)
R      0    79   0        (0.0/79.0)
Total  87   81   0.0119   (2.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.497735     0.9875    80
max f2                       0.497735     0.994962  80
max f0point5                 0.654627     0.992167  75
max accuracy                 0.497735     0.988095  80
max precision                0.999951     1         0
max recall                   0.497735     1         80
max specificity              0.999951     1         0
max absolute_mcc             0.497735     0.976418  80
max min_per_class_accuracy   0.56032      0.977528  79
max mean_per_class_accuracy  0.497735     0.988764  80
max tns                      0.999951     89        0
max fns                      0.999951     78        0
max fps                      5.94512e-06  89        167
max tps                      0.497735     79        80
max tnr                      0.999951     1         0
max fnr                      0.999951     0.987342  0
max fpr                      5.94512e-06  1         167
max tpr                      0.497735     1         80

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 47.40 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999833           2.12658  2.12658            1                0.999898     1                           0.999898            0.0253165       0.0253165              

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [100,100,100], cross folds = 5, activation function = "tanh"

In [19]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[100,100,100], nfolds=5, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1547


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,502 weights/biases, 329.9 KB, 840 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ---------------------  -------------------  -----------------------  --------------------
    1        60       Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.005349432434537448  0.0024700164794921875  0.0         0.0033385474209062524  0.11173787713050842  0.0032514846402079594    0.019395649433135986
    3        100      Tanh     0.0        0.0   0.0   0.00954733757362701   0.005054263398051262   0.0         0.0007284714569662355  0.10186704993247986  -0.0005936582011547994   0.0332607626914978
    4        100      Tanh     0.0        0.0   0.0   0.03129991335116793   0.06000445783138275    0.0         0.0011701016590680866  0.10121071338653564  -0.0018265317456253496   0.022572875022888184
    5        2        Softmax             0.0   0.0   0.004927999314386398  0.0016948780976235867  0.0         -0.007474573575891554  0.5437984466552734   -1.3877787807814457e-17  0.02156270295381546

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.009526562350004828
RMSE: 0.09760411031306432
LogLoss: 0.047159162656063146
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4058787225176554
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      89   0    0        (0.0/89.0)
R      0    79   0        (0.0/79.0)
Total  89   79   0        (0.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.405879     1         78
max f2                       0.405879     1         78
max f0point5                 0.405879     1         78
max accuracy                 0.405879     1         78
max precision                0.999997     1         0
max recall                   0.405879     1         78
max specificity              0.999997     1         0
max absolute_mcc             0.405879     1         78
max min_per_class_accuracy   0.405879     1         78
max mean_per_class_accuracy  0.405879     1         78
max tns                      0.999997     89        0
max fns                      0.999997     78        0
max fps                      3.02536e-08  89        167
max tps                      0.405879     79        78
max tnr                      0.999997     1         0
max fnr                      0.999997     0.987342  0
max fpr                      3.02536e-08  1         167
max tpr                      0.405879     1         78

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 45.88 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999971           2.12658  2.12658            1                0.999997     1                           0.999997            0.0253165       0.0253165                  112.658  112.658            0.0253165
2        0.0238095       

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [5,5,5], cross folds = 10, activation function = "tanh"

In [20]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[5,5,5], nfolds=10, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1646


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 377 weights/biases, 20.4 KB, 1,848 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias             bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  --------------------  --------------------
    1        60       Input    0.0
    2        5        Tanh     0.0        0.0   0.0   0.0027169045708918325  0.0015208953991532326  0.0         -0.023698948222057273  0.18388843536376953  0.015306471072023042  0.06186409294605255
    3        5        Tanh     0.0        0.0   0.0   0.004841180955991149   0.004191089421510696   0.0         -0.03935050696134567   0.46970808506011963  0.0319510136990834    0.049038395285606384
    4        5        Tanh     0.0        0.0   0.0   0.02862795041175559    0.0436774343252182     0.0         0.04336357787251473    0.40053725242614746  0.05858446937420161   0.11086133122444153
    5        2        Softmax             0.0   0.0   0.003637024899944663   0.0012923544272780418  0.0         1.6811173975467681     0.9655880928039551   2.42861286636753e-17  0.03753940761089325

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.14209526860272684
RMSE: 0.3769552607441987
LogLoss: 0.4472297131145348
Mean Per-Class Error: 0.1792063717821078
AUC: 0.8718532214478737
AUCPR: 0.8779719303544561
Gini: 0.7437064428957474

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4606250882069269
       M    R    Error    Rate
-----  ---  ---  -------  ------------
M      74   15   0.1685   (15.0/89.0)
R      15   64   0.1899   (15.0/79.0)
Total  89   79   0.1786   (30.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.460625     0.810127  78
max f2                       0.223516     0.850575  118
max f0point5                 0.668388     0.825083  55
max accuracy                 0.460625     0.821429  78
max precision                0.926492     1         0
max recall                   0.100605     1         161
max specificity              0.926492     1         0
max absolute_mcc             0.460625     0.641587  78
max min_per_class_accuracy   0.460625     0.810127  78
max mean_per_class_accuracy  0.460625     0.820794  78
max tns                      0.926492     89        0
max fns                      0.926492     78        0
max fps                      0.0874641    89        167
max tps                      0.100605     79        161
max tnr                      0.926492     1         0
max fnr                      0.926492     0.987342  0
max fpr                      0.0874641    1         167
max tpr                      0.100605     1         161

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 46.78 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score     cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain       cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  --------  --------------------------  ------------------  --------------  -------------------------  ---------  -----------------  --------------------
1        0.0119048                   0.919765           2.12658   2.12658            1                0.924116  1                           0.924116            0.0253165       0.0253165                  112.658    112.

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "tanh"


In [21]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[20,20,20], nfolds=10, activation="tanh")

deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_1922


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 2,102 weights/biases, 41.1 KB, 1,848 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate              rate_rms               momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  ---------------------  ---------------------  ----------  ---------------------  -------------------  -----------------------  ----------------------
    1        60       Input    0.0
    2        20       Tanh     0.0        0.0   0.0   0.003291342377876087   0.0015660692006349564  0.0         0.0005457722448287919  0.16605037450790405  -0.005584892241281153    0.039226993918418884
    3        20       Tanh     0.0        0.0   0.0   0.003952196570171509   0.002457999624311924   0.0         0.002471340746851638   0.2246662974357605   0.0007519421823126811    0.04899381101131439
    4        20       Tanh     0.0        0.0   0.0   0.0032213299757859206  0.00198948010802269    0.0         0.010227235884522089   0.21523654460906982  0.0003222132047618339    0.028056152164936066
    5        2        Softmax             0.0   0.0   0.0038025165675207974  0.0009930464439094067  0.0         0.036994841787964104   1.1569404602050781   -2.2768245622195593e-18  5.2304385462775826e-05

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.019154250190460676
RMSE: 0.13839888074135814
LogLoss: 0.09026942167032995
Mean Per-Class Error: 0.006329113924050633
AUC: 0.9992888636040392
AUCPR: 0.9992420132248914
Gini: 0.9985777272080785

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6477317164213503
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      89   0    0        (0.0/89.0)
R      1    78   0.0127   (1.0/79.0)
Total  90   78   0.006    (1.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.647732     0.993631  77
max f2                       0.647732     0.989848  77
max f0point5                 0.647732     0.997442  77
max accuracy                 0.647732     0.994048  77
max precision                0.999951     1         0
max recall                   0.371238     1         83
max specificity              0.999951     1         0
max absolute_mcc             0.647732     0.988115  77
max min_per_class_accuracy   0.647732     0.987342  77
max mean_per_class_accuracy  0.647732     0.993671  77
max tns                      0.999951     89        0
max fns                      0.999951     78        0
max fps                      4.97155e-06  89        167
max tps                      0.371238     79        83
max tnr                      0.999951     1         0
max fnr                      0.999951     0.987342  0
max fpr                      4.97155e-06  1         167
max tpr                      0.371238     1         83

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 48.24 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999772           2.12658  2.12658            1                0.99989      1                           0.99989             0.0253165       0.0253165        

None


Params dataset = iris, features = ["sepal length", "sepal width", "petal length", "petal width"], target = "class", hidden layers = [20,20,20], cross folds = 10, activation function = "tanh"


In [22]:
run_experiment(dataset=sonar, features=features, target=target, hidden=[100,100,100], nfolds=10, activation="tanh")


deeplearning Model Build progress: |█████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ODeepLearningEstimator : Deep Learning
Model Key: DeepLearning_model_python_1748027061198_2204


Status of Neuron Layers: predicting class, 2-class classification, bernoulli distribution, CrossEntropy loss, 26,502 weights/biases, 329.9 KB, 840 training samples, mini-batch size 1
    layer    units    type     dropout    l1    l2    mean_rate             rate_rms               momentum    mean_weight            weight_rms           mean_bias                bias_rms
--  -------  -------  -------  ---------  ----  ----  --------------------  ---------------------  ----------  ---------------------  -------------------  -----------------------  --------------------
    1        60       Input    0.0
    2        100      Tanh     0.0        0.0   0.0   0.005349432434537448  0.0024700164794921875  0.0         0.0033385474209062524  0.11173787713050842  0.0032514846402079594    0.019395649433135986
    3        100      Tanh     0.0        0.0   0.0   0.00954733757362701   0.005054263398051262   0.0         0.0007284714569662355  0.10186704993247986  -0.0005936582011547994   0.0332607626914978
    4        100      Tanh     0.0        0.0   0.0   0.03129991335116793   0.06000445783138275    0.0         0.0011701016590680866  0.10121071338653564  -0.0018265317456253496   0.022572875022888184
    5        2        Softmax             0.0   0.0   0.004927999314386398  0.0016948780976235867  0.0         -0.007474573575891554  0.5437984466552734   -1.3877787807814457e-17  0.02156270295381546

ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.009526562350004828
RMSE: 0.09760411031306432
LogLoss: 0.047159162656063146
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4058787225176554
       M    R    Error    Rate
-----  ---  ---  -------  -----------
M      89   0    0        (0.0/89.0)
R      0    79   0        (0.0/79.0)
Total  89   79   0        (0.0/168.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.405879     1         78
max f2                       0.405879     1         78
max f0point5                 0.405879     1         78
max accuracy                 0.405879     1         78
max precision                0.999997     1         0
max recall                   0.405879     1         78
max specificity              0.999997     1         0
max absolute_mcc             0.405879     1         78
max min_per_class_accuracy   0.405879     1         78
max mean_per_class_accuracy  0.405879     1         78
max tns                      0.999997     89        0
max fns                      0.999997     78        0
max fps                      3.02536e-08  89        167
max tps                      0.405879     79        78
max tnr                      0.999997     1         0
max fnr                      0.999997     0.987342  0
max fpr                      3.02536e-08  1         167
max tpr                      0.405879     1         78

Gains/Lift Table: Avg response rate: 47.02 %, avg score: 45.88 %
group    cumulative_data_fraction    lower_threshold    lift     cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain     cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  -------  -----------------  --------------------
1        0.0119048                   0.999971           2.12658  2.12658            1                0.999997     1                           0.999997            0.0253165       0.0253165                  112.658  112.658            0.0253165
2        0.0238095       

None


In [86]:
from tabulate import tabulate
from IPython.display import Markdown, display, HTML
headers = ["Activation Function", "Cross Validation", "Hidden Layers", "Training Error %", "Validation Error %", "Cross Validation Mean Error %", "Training Time"]
#print(metrics)
print(tabulate(metrics, headers=headers, tablefmt="grid"))

+-----------------------+--------------------+-----------------+---------------------+----------------------+---------------------------------+-----------------+
| Activation Function   | Cross Validation   | Hidden Layers   | Training Error %    | Validation Error %   | Cross Validation Mean Error %   | Training Time   |
+=======================+====================+=================+=====================+======================+=================================+=================+
| rectifier             | None               | [5, 5, 5]       | 27.471198975963596% | 29.292929292929294%  | -                               | 0.393 s         |
+-----------------------+--------------------+-----------------+---------------------+----------------------+---------------------------------+-----------------+
| rectifier             | None               | [20, 20, 20]    | 2.2471910112359605% | 14.141414141414144%  | -                               | 0.461 s         |
+-----------------------+---